# 07-04

## Challenge 05 - Spark Streaming
---
![](https://images.unsplash.com/photo-1531254003608-7197414b0154?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1350&q=80)
Picture by [Gavin Barnett](https://unsplash.com/photos/SPgm3N9AjcA)

- Now that you have covered a lot of PySpark, let's have a look at a specific Spark library `Spark Streaming`.
- Like we saw in the course, Spark Streaming is a library for near-real-time processing. It actually works with micro-batches of data.

In this challenge we will setup a Spark Streaming application which counts words coming from a server. We will create the app for processing, and we will open a simple server with netcat. [Here](https://www.geeek.org/netcat-l-outil-le-plus-indispensable-apres-ls-cd-rm/) you can find out more about the netcat util.

- **<font color='red'>Please read this carefully</font>**

This challenge **will not run in the notebook** : you will have to create a .py file and execute it using the spark-submit command, to be interpreted by spark.

- For each step below, you need to understand what the code is doing. 
- Links to documentation are included to help you.
- Each code cell below is to be added into a **network_wordcount.py** that will be run at the end

### Calling the StreamingContext

First, we import StreamingContext, which is the main entry point for all streaming functionality. We create a local StreamingContext with two execution threads, and batch interval of 1 second.

Once you have a **StreamingContext**, which is a **class instance**, you have access to all its methods : [Have a look at the doc](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext)

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second

### Designate the data source

Using this context, we can **create a DStream** that represents streaming data from a TCP source, specified as hostname (e.g. localhost) and port (e.g. 9999).

Note that the *ssc.socketTextStream()* in one method you have seen in the previous documentation link of [StreamingContext](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.socketTextStream).

##### Doc extract :

```
Create an input from TCP source hostname:port. Data is received using a TCP socket and receive byte is interpreted as UTF8 encoded \n delimited lines.
```

In [ ]:
# Create a DStream that will connect to hostname:port, like localhost:9999

### Process the lines

##### Split lines by space

This lines DStream represents the stream of data that will be received from the data server. Each record in this DStream is a line of text. Next, we want to split the lines by space into words.

##### map and count words

flatMap is a one-to-many DStream operation that creates a new DStream by generating multiple new records from each record in the source DStream. In this case, each line will be split into multiple words and the stream of words is represented as the words DStream. Next, we want to count these words.

![Dstreams](https://spark.apache.org/docs/latest/img/streaming-dstream-ops.png)

In [ ]:
# Split each line into words

### Count and display

The words DStream is further mapped (one-to-one transformation) to a DStream of (word, 1) pairs, which is then reduced to get the frequency of words in each batch of data. Finally, wordCounts.pprint() will print a few of the counts generated every second.

Note that when these lines are executed, Spark Streaming only sets up the computation it will perform when it is started, and no real processing has started yet. To start the processing after all the transformations have been setup, we finally call :

In [ ]:
# Count each word in each batch

# Print the first ten elements of each RDD generated in this DStream to the console

### Start computation

In [ ]:
# Start the computation

# Wait for the computation to terminate 

**The complete code should look like this :**

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second

# Create a DStream that will connect to hostname:port, like localhost:9999

# Split each line into words

# Count each word in each batch

# Print the first ten elements of each RDD generated in this DStream to the console

 # Start the computation
 # Wait for the computation to terminate

If you have already downloaded and built Spark, you can run this example as follows. You will first need to run Netcat (a small utility found in most Unix-like systems) as a data server by using :

```
# TERMINAL 1:
# Running Netcat

$ nc -lk 9999
> Hello world
> Type anything you want
> Can you type 3 messages in less than 1 second ?
> Typing party !
```

```
# TERMINAL 2: RUNNING network_wordcount.py

$ ./spark-2.4.3-bin-hadoop2.7/bin/spark-submit your/path/network_wordcount.py localhost 9999
...
-------------------------------------------
Time: 2014-10-14 15:25:21
-------------------------------------------
(Hello,1)
(world,1)
(Type,1)
(anything,1)
(you,1)
(want,1)
...
```

### Wow, too much INFO

##### Problem 

When you launch your python script with spark-submit, you should get a lot of INFO messages in your TERMINAL2. This is absolutely normal, INFO logs are not errors, it's normal operation. But here, **we have so much INFO messages that we can't even read our script results**. Did you notice that ?

##### Solution

As always, [somebody else already had this issue on stackOverflow](https://stackoverflow.com/questions/25193488/how-to-turn-off-info-logging-in-spark) : This solution goes into *conf/log4j.properties*, the global configuration file for Spark.

Another solution is to add a line into your code, so that you don't edit properties for later. What you want to do is to tell Spark to calm down and display only important messages in the console output.

```sc.setLogLevel("ERROR")```

You have to add this line in your code, preferably just after defining the **sc** variable.